In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.seasonal import STL


In [2]:
# list_data_name = ["BTCUSDT_1d", "ETHUSDT_1d", "BNBUSDT_1d", "XRPUSDT_1d", "ADAUSDT_1d", "LTCUSDT_1d"]                   
# list_data_name = ["BTCUSDT_1h", "ETHUSDT_1h", "BNBUSDT_1h", "XRPUSDT_1h", "ADAUSDT_1h", "LTCUSDT_1h"]                   

list_data_name = ["ADAUSDT_1h"]                   

from_date = '2021-12-31 00:00:00' # 2020-12-31 00:00:00
until_date = '2025-07-31 00:00:00'
column_name = 'processed_log_return_wtmra_0'

In [3]:
for data_name in list_data_name:
    # Importing data
    data = pd.read_csv(f'../data/01-output-{data_name}-from-{from_date}-until-{until_date}-log-return.csv', parse_dates=['date'], index_col='date')
    data = data[[column_name]]
    # Assuming `data` is your hourly time series data
    fft_result = np.fft.fft(data)
    frequencies = np.fft.fftfreq(len(data), d=1)  # d is the sample spacing, 1 hour in this case    

    # Identifying dominant frequency and converting to period
    dominant_frequency = abs(frequencies[np.argmax(np.abs(fft_result))])
    dominant_period = 1 / dominant_frequency  # Convert to hours per cycle

    stl = STL(data[column_name], period=24, robust=True)
    res = stl.fit()

    # Add the trend, seasonality and residual to the original data
    data['processed_log_return_wtmra_0_trend'] = res.trend
    data['processed_log_return_wtmra_0_trend_seasonal'] = res.seasonal
    data['processed_log_return_wtmra_0_trend_residual'] = res.resid    

    # remove the nan values
    data = data.dropna()    

    # Exporting data
    data.to_csv(f'../data/02d-output-{data_name}-log-return-temporal_decomposition.csv')